In [ ]:
#Run this cell if on colab
!apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip libboost-all-dev

!pip install vizdoom


In [8]:
def make_vid(frames):
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter('./gameplay.avi', fourcc, 25, (frames[0].shape[1], frames[0].shape[0]), True)
    for frame in frames:
        writer.write(frame)
    writer.release()

In [2]:
from vizdoom import *
import random
import time
import cv2
import numpy as np
from model import Agent
import torch